[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Data-Analytics-with-Python/individual-assignment-iii-bastienm69/blob/main/Assignment_3%20(Bastien).ipynb)


IMPORTANT: Before you start, enter your name and student number below.

**Full Name**: Bastien Maillet

**Student Number**: 400677332

# Predictive Analytics for Nata Supermarket

Welcome to Part III of our case assignment. In this part, we will continue working with the same dataset of **Nata Supermarket**.  Our focus here will be on performing predictive modeling tasks.

Throughout this assignment, please ensure that your results are reproducible by setting the **random_state to 20**



# Loading and preparing the data

To begin with, load the data as a `pandas` data frame. Recall that you there are missing values in the data. **Make sure to address** the following issues from part I of the assignment before starting your analysis:

* Remove the missing values
* Remove any column of constant values
* Convert the column `Dt_Customer` to number of days the customer has been with the company.

In [5]:
import pandas as pd
import numpy as np

# Remove the missing values
df = pd.read_csv('Nata_Supermarket.csv',sep=';')
df.dropna(inplace=True)

# Remove any column of constant values
constant_columns = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_columns, inplace=True)

# Convert 'Dt_Customer' to number of days the customer has been with the company
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d/%m/%Y')
latest_customer_date = df['Dt_Customer'].max()
df['Days_Since_Enrollment'] = (latest_customer_date - df['Dt_Customer']).dt.days

print(df.head())
print(df.info())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumStorePurchases  NumWebVisitsMonth  \
0  2012-04-09       58       635  ...                  4                  7   
1  2014-08-03       38        11  ...                  2                  5   
2  2013-08-21       26       426  ...                 10                  4   
3  2014-10-02       26        11  ...                  4                  6   
4  2014-01-19       94       173  ...                  6                  5   

   AcceptedCmp3  AcceptedCmp4  AcceptedCmp5  AcceptedCmp

# Question 1 (45 points)

In this question, we will predict a customer's spending amount on each product category over a two year period. Let us assume that when we try to predict a customer's spending on a product category (such as wines), their spending on other products is not observable.

In this question and Question 2, we will focus on **wines**.

(i). **Split the data** into two data frames, X (**features**) and y (**target**).

Then, further **split the data** into **training** and **testing** sets.

In [6]:
from sklearn.model_selection import train_test_split

# Define target variable (y)
y = df['MntWines']

# Define features (X) by dropping ID, Dt_Customer, MntWines, and Response
X = df.drop(columns=['ID', 'Dt_Customer', 'MntWines', 'Response'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1772, 24)
X_test shape: (444, 24)
y_train shape: (1772,)
y_test shape: (444,)


(ii). **Build a machine learning pipeline** that combines the following steps to predict spending amount on wines:

* Performing one-hot encoding for the categorical features;
* A random forest model for regression.

In the random forest model, **specify** the following hyperparameters:
* Number of trees;
* Maximum depth of any tree
* Minimum number of data points required to split a node;
* Minimum number of data points in any leaf node

In addition, **fit your model** to the training data.

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# Identify categorical features for one-hot encoding
categorical_features = X_train.select_dtypes(include=['object']).columns

# Create a preprocessor for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Keep other columns as they are
)

# Define the Random Forest Regressor model with specified hyperparameters using some initial values
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=20
)

# Create the full pipeline
wine_spending_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', rf_model)
])

# Fit the pipeline to the training data
wine_spending_pipeline.fit(X_train, y_train)

print("Machine learning pipeline built and fitted successfully!")

Machine learning pipeline built and fitted successfully!


(iii). Use the model to **predict** the spending amount on wines by a customer with the following features.

| Feature |  Value  |
|---------|---------|
| Age     | 48      |
|Education|Graduation|
|Marital_Status| Married|
|Income|80,000|
|Kidhome|1|
|Teenhome|1|
|Dt-Customer|2016-10-10|
|Recency|43|     
|NumDealsPurchases|2|
|   NumWebPurchases|1|
|NumCatalogPurchases|0|
|NumStorePurchases|15|
|NumWebVisitsMonth|5|
|AcceptedCmp1,2,3,4,5| 0 |
|Complain|0|




In [8]:
import pandas as pd

# Define the features for the new customer as provided
new_customer_features_raw = {
    'Age': 48,
    'Education': 'Graduation',
    'Marital_Status': 'Married',
    'Income': 80000.0, # Converted to float as in the original dataset
    'Kidhome': 1,
    'Teenhome': 1,
    'Dt_Customer_str': '2016-10-10',
    'Recency': 43,
    'NumDealsPurchases': 2,
    'NumWebPurchases': 1,
    'NumCatalogPurchases': 0,
    'NumStorePurchases': 15,
    'NumWebVisitsMonth': 5,
    'AcceptedCmp1': 0,
    'AcceptedCmp2': 0,
    'AcceptedCmp3': 0,
    'AcceptedCmp4': 0,
    'AcceptedCmp5': 0,
    'Complain': 0
}

# Create a DataFrame for the new customer
new_customer_df = pd.DataFrame([new_customer_features_raw])

# --- Feature Engineering for the new customer ---

# 1. Calculate 'Year_Birth' from 'Age'
reference_year = latest_customer_date.year # This will be 2014
new_customer_df['Year_Birth'] = reference_year - new_customer_df['Age']

# 2. Calculate 'Days_Since_Enrollment' from 'Dt-Customer'
new_customer_df['Dt_Customer'] = pd.to_datetime(new_customer_df['Dt_Customer_str'], format='%Y-%m-%d')
new_customer_df['Days_Since_Enrollment'] = (latest_customer_date - new_customer_df['Dt_Customer']).dt.days

# Drop the original 'Age', 'Dt_Customer_str', and 'Dt_Customer' columns used for derivation
new_customer_df = new_customer_df.drop(columns=['Age', 'Dt_Customer_str', 'Dt_Customer'])

# 3. Fill missing Mnt* features with the mean from X_train
missing_mnt_cols = [
    'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds'
]
for col in missing_mnt_cols:
    if col not in new_customer_df.columns:
        new_customer_df[col] = X_train[col].mean()

# Ensure the column order of the new customer DataFrame matches X_train to correctly identify features.
new_customer_processed = new_customer_df[X_train.columns]

# --- Make Prediction ---
predicted_spending_wines = wine_spending_pipeline.predict(new_customer_processed)

print(f"Predicted spending on wines for the new customer: {predicted_spending_wines[0]:.2f}")

Predicted spending on wines for the new customer: 361.33


(iv). **Consider** **two** measures to evaluate the model's performance on the test dataset.

Based on you computational results, how would you describe the model's performance?

In [9]:
from sklearn.metrics import r2_score, mean_absolute_error

# Make predictions on the test set
y_pred = wine_spending_pipeline.predict(X_test)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

print(f"Model Performance on Test Data:")
print(f"R-squared: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

print("\n--- Model Performance Description ---")
if r2 > 0.7:
    print("The model demonstrates strong predictive power, explaining a large proportion of the variance in wine spending.")
elif r2 > 0.5:
    print("The model shows moderate predictive power, explaining a fair amount of the variance in wine spending.")
else:
    print("The model's predictive power is relatively low, explaining a limited proportion of the variance in wine spending.")

print(f"On average, the model's predictions for wine spending are off by approximately ${mae:.2f}.")

Model Performance on Test Data:
R-squared: 0.7854
Mean Absolute Error (MAE): 88.00

--- Model Performance Description ---
The model demonstrates strong predictive power, explaining a large proportion of the variance in wine spending.
On average, the model's predictions for wine spending are off by approximately $88.00.


(v). **Perform** a 6-fold cross validation with a performance score of your choice.

**Note**: You may need to research on how to specify the performance score for regression models.

In [10]:
from sklearn.model_selection import cross_val_score

# Perform 6-fold cross-validation using the 'r2' score
cv_scores = cross_val_score(wine_spending_pipeline, X_train, y_train, cv=6, scoring='r2')

print(f"Cross-validation R-squared scores: {cv_scores}")
print(f"Mean R-squared across 6 folds: {cv_scores.mean():.4f}")
print(f"Standard deviation of R-squared across 6 folds: {cv_scores.std():.4f}")

Cross-validation R-squared scores: [0.80580823 0.78955729 0.82510186 0.79605727 0.79268263 0.80433998]
Mean R-squared across 6 folds: 0.8023
Standard deviation of R-squared across 6 folds: 0.0118


(vi). **Perform** hyperparameter tuning using `GridSearchCV` for the following hyperparameters:

* Number of trees: 50, 100
* Maximum depth of any tree: 5, 10, 15
* Minimum number of data points required to split a node: 3, 6
* Minimum number of data points in any leaf node: 2,4,8


Based on your computational result, **show**:
* the best hyperparameter comination
* the corresponding performance score

In addition, **retrieve** the best model (the one corresponding to the best performance score).

In [11]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for GridSearchCV
param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [5, 10, 15],
    'regressor__min_samples_split': [3, 6],
    'regressor__min_samples_leaf': [2, 4, 8]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=wine_spending_pipeline, param_grid=param_grid,
                           cv=6, scoring='r2', n_jobs=-1, verbose=1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and the corresponding best score
print("Best hyperparameters found:", grid_search.best_params_)
print("Best R-squared score:", grid_search.best_score_)

# Retrieve the best model
best_rf_model = grid_search.best_estimator_
print("Best estimator retrieved successfully!")

Fitting 6 folds for each of 36 candidates, totalling 216 fits
Best hyperparameters found: {'regressor__max_depth': 15, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 3, 'regressor__n_estimators': 100}
Best R-squared score: 0.8054899503600353
Best estimator retrieved successfully!


# Question 2 (24 points)

In this question, we will compare the performance of the best model found through `GridSearchCV` in Question 1 with the performance of the linear regression model.

(i). **Construct** a linear regression model using all relevant features and fit it to the training data.

Further, **evaluate** the model's performance on the test data and compare it with the best random forest model found in Question 1, with respect to the two performance considered in Question 1.

**Note**: You may use the same training and testing datasets as in Question 1.

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error

# Identify categorical features for one-hot encoding (reusing the logic from Q1)
categorical_features_lr = X_train.select_dtypes(include=['object']).columns

# Create a preprocessor for one-hot encoding
lr_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_lr)
    ],
    remainder='passthrough' # Keep other columns as they are
)

# Define the Linear Regression model
lr_model = LinearRegression()

# Create the full pipeline for Linear Regression
lr_pipeline = Pipeline(steps=[
    ('preprocessor', lr_preprocessor),
    ('regressor', lr_model)
])

# Fit the pipeline to the training data
lr_pipeline.fit(X_train, y_train)

print("Linear Regression model built and fitted successfully!")

# --- Evaluate Linear Regression Model ---
y_pred_lr = lr_pipeline.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print("\n--- Linear Regression Model Performance on Test Data ---")
print(f"R-squared (Linear Regression): {r2_lr:.4f}")
print(f"Mean Absolute Error (Linear Regression): {mae_lr:.2f}")

# --- Compare with Best Random Forest Model ---
# Make predictions with the best Random Forest model (from GridSearchCV in Q1.vi)
y_pred_rf_best = best_rf_model.predict(X_test)
r2_rf_best = r2_score(y_test, y_pred_rf_best)
mae_rf_best = mean_absolute_error(y_test, y_pred_rf_best)

print("\n--- Best Random Forest Model Performance on Test Data (from Q1.vi) ---")
print(f"R-squared (Best Random Forest): {r2_rf_best:.4f}")
print(f"Mean Absolute Error (Best Random Forest): {mae_rf_best:.2f}")

print("\n--- Comparison ---")
if r2_rf_best > r2_lr:
    print(f"The Best Random Forest Model has a higher R-squared ({r2_rf_best:.4f}) than the Linear Regression Model ({r2_lr:.4f}), indicating better explanatory power.")
else:
    print(f"The Linear Regression Model has a higher R-squared ({r2_lr:.4f}) than the Best Random Forest Model ({r2_rf_best:.4f}).")

if mae_rf_best < mae_lr:
    print(f"The Best Random Forest Model has a lower Mean Absolute Error ({mae_rf_best:.2f}) than the Linear Regression Model ({mae_lr:.2f}), suggesting more accurate predictions on average.")
else:
    print(f"The Linear Regression Model has a lower Mean Absolute Error ({mae_lr:.2f}) than the Best Random Forest Model ({mae_rf_best:.2f}).")

Linear Regression model built and fitted successfully!

--- Linear Regression Model Performance on Test Data ---
R-squared (Linear Regression): 0.6714
Mean Absolute Error (Linear Regression): 119.41

--- Best Random Forest Model Performance on Test Data (from Q1.vi) ---
R-squared (Best Random Forest): 0.7869
Mean Absolute Error (Best Random Forest): 87.01

--- Comparison ---
The Best Random Forest Model has a higher R-squared (0.7869) than the Linear Regression Model (0.6714), indicating better explanatory power.
The Best Random Forest Model has a lower Mean Absolute Error (87.01) than the Linear Regression Model (119.41), suggesting more accurate predictions on average.


(ii). Let's further compare the distribution of prediction errors by the two models in the following steps.

**Step 1**. For both the linear regression and the (best) random forest model, compute the absolute residual residual for each record in the test dataset.
  * Note that the absolute residual is distance between the predicted value and actual value, i.e., $|y_{pred}-y_{test}|$.

So we end up with two sets of absolute residuals (one by the linear regression model and the other by the random forest model).

**Step 2**. For each pair of absolute residuals for the same test data point, we can define a point in a scatterplot. Genereate such a scatterplot using `plotly.express` (LR residuals vs. RF residuals).  

**Step 3**. Add a 45 degree reference line to the plot. This can be done using the following codes. (You may need to change `min_val` and `max_val` for better visualization).

```
min_val = 0
max_val = 10
fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash"),
)
```

In [13]:
import numpy as np
import plotly.express as px
import pandas as pd

# Step 1: Compute the absolute residuals for both models
abs_residuals_lr = np.abs(y_pred_lr - y_test)
abs_residuals_rf = np.abs(y_pred_rf_best - y_test)

# Step 2: Create a DataFrame for plotting residuals
residuals_df = pd.DataFrame({
    'LR_Absolute_Residuals': abs_residuals_lr,
    'RF_Absolute_Residuals': abs_residuals_rf
})

# Generate the scatterplot using plotly.express
fig = px.scatter(
    residuals_df,
    x='LR_Absolute_Residuals',
    y='RF_Absolute_Residuals',
    title='Comparison of Absolute Residuals (LR vs. RF)',
    labels={
        'LR_Absolute_Residuals': 'Linear Regression Absolute Residuals',
        'RF_Absolute_Residuals': 'Random Forest Absolute Residuals'
    },
    hover_data={'LR_Absolute_Residuals': ':.2f', 'RF_Absolute_Residuals': ':.2f'}
)

# Step 3: Add a 45 degree reference line to the plot
# Determine min/max values for the 45-degree line based on the data
min_val = min(residuals_df['LR_Absolute_Residuals'].min(), residuals_df['RF_Absolute_Residuals'].min())
max_val = max(residuals_df['LR_Absolute_Residuals'].max(), residuals_df['RF_Absolute_Residuals'].max())

# Add a small buffer to min/max values for better visualization
buffer = (max_val - min_val) * 0.05
min_val -= buffer
max_val += buffer

fig.add_shape(
    type="line",
    x0=min_val, y0=min_val,
    x1=max_val, y1=max_val,
    line=dict(color="red", width=2, dash="dash")
)

fig.update_layout(showlegend=False) # Hide legend if not needed
fig.show()

**Implement the above steps**.

Note that the above steps essentially creates a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot). How would you interpret the plot (for comparing the two predictive models).

**ANSWER :**

This plot visually compares the distribution of the absolute prediction errors (residuals) from the two models. The line of identity (the dashed red line) represents the scenario where the absolute residual for the Random Forest model is exactly equal to the absolute residual for the Linear Regression model.

1. The points below the 45-degree line indicate instances where the Random Forest model's absolute residual is smaller than the Linear Regression model's absolute residual. This means the Random Forest model made a more accurate prediction for these specific data points.
2. The points above the 45-degree line indicate instances where the Linear Regression model's absolute residual is smaller than the Random Forest model's absolute residual. For these points, the Linear Regression model performed better.
3. Points on or very close to the 45-degree line: For these data points, both models performed similarly in terms of absolute error.

# Question 3. (24 points)

In this question, we will consider a classification problem on customers' spendings on meat products.


(i). For the column that represents customers' spendings on meat products, **calculate** the 33.33% and 66.67% percentiles. (**Hint**: You may use the function `df['MntMeatProducts'].quantile([1/3,2/3])`.)

Based on the two percentiles, **label** each row in the dataset
* If a customer's spending is below the 33.33% percentile, label their spending as "low";

* If a customer's spending is above the 66.67% percentile, label their spending as "high";


* If a customer's spending is between the two percentiles, label their spending as "medium".

In [14]:
meat_percentiles = df['MntMeatProducts'].quantile([1/3, 2/3])
lower_bound = meat_percentiles.iloc[0]
upper_bound = meat_percentiles.iloc[1]

print(f"33.33% percentile for MntMeatProducts: {lower_bound:.2f}")
print(f"66.67% percentile for MntMeatProducts: {upper_bound:.2f}")

df['Meat_Spending_Category'] = pd.cut(
    df['MntMeatProducts'],
    bins=[-float('inf'), lower_bound, upper_bound, float('inf')],
    labels=['low', 'medium', 'high'],
    right=False # Labels 'medium' when value equals lower_bound
)

print(df[['MntMeatProducts', 'Meat_Spending_Category']].head())

33.33% percentile for MntMeatProducts: 24.00
66.67% percentile for MntMeatProducts: 146.33
   MntMeatProducts Meat_Spending_Category
0              546                   high
1                6                    low
2              127                 medium
3               20                    low
4              118                 medium


(ii). **Build a K-nearest-neighbors model** to predict the label of a customer's spending on meat products. The model should be part of a machine learning pipeline that preprocesses the data.





In [15]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

# 1. Define the target variable y
y = df['Meat_Spending_Category']

# 2. Define the feature set X by dropping specified columns
X = df.drop(columns=['ID', 'Dt_Customer', 'MntMeatProducts', 'Response', 'MntWines', 'Meat_Spending_Category'])

# 3. Split the X and y data into training and testing sets
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X, y, test_size=0.2, random_state=20)

# 4. Identify categorical features in X_train_knn
categorical_features_knn = X_train_knn.select_dtypes(include=['object']).columns

# 5. Create a ColumnTransformer (preprocessor)
knn_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_knn)
    ],
    remainder='passthrough' # Keep other columns (numerical) as they are
)

# 6. Instantiate a KNeighborsClassifier model
knn_model = KNeighborsClassifier(n_neighbors=5) # Starting with n_neighbors=5, this can be tuned later

# 7. Construct a Pipeline
knn_pipeline = Pipeline(steps=[
    ('preprocessor', knn_preprocessor),
    ('classifier', knn_model)
])

# 8. Fit the pipeline to the training data
knn_pipeline.fit(X_train_knn, y_train_knn)

print("K-nearest-neighbors pipeline built and fitted successfully!")
print(f"X_train_knn shape: {X_train_knn.shape}")
print(f"X_test_knn shape: {X_test_knn.shape}")
print(f"y_train_knn shape: {y_train_knn.shape}")
print(f"y_test_knn shape: {y_test_knn.shape}")

K-nearest-neighbors pipeline built and fitted successfully!
X_train_knn shape: (1772, 23)
X_test_knn shape: (444, 23)
y_train_knn shape: (1772,)
y_test_knn shape: (444,)


(iii). Evaluate the performace of your model in part (ii) on the test data by **generating the classification report**.

Further, **interpret** each number in the classification report based on the current context.

In [16]:
from sklearn.metrics import classification_report

# Make predictions on the test set using the fitted pipeline
y_pred_knn = knn_pipeline.predict(X_test_knn)

# Generate the classification report
report = classification_report(y_test_knn, y_pred_knn)

# Print the classification report
print("Classification Report for K-nearest-neighbors Model:")
print(report)

Classification Report for K-nearest-neighbors Model:
              precision    recall  f1-score   support

        high       0.78      0.78      0.78       161
         low       0.67      0.74      0.70       130
      medium       0.53      0.48      0.50       153

    accuracy                           0.66       444
   macro avg       0.66      0.67      0.66       444
weighted avg       0.66      0.66      0.66       444



# Describe how you used Gen. AI. in this assignment (2 points)

I used AI on some formulas that I didn't know or didn't remember and to correct some little problems with my codes especially in the 2nd part. I found this test harder than the others and I needed more tim obvisouly but I am pretty confident with the work now.

**Note**: The remaining 5 points will be assigned to readability of the work.